In [1]:
from ray import tune
import numpy as np
import pandas as pd
import os
import datetime
from ray.tune.suggest.bayesopt import BayesOptSearch
from one_region_simu import *

/srv/main-spack-instance/spack/var/spack/environments/release_v0_1_202109_hotfix/.spack-env/view/lib/python3.8/site-packages/tvb/datatypes/surfaces.py:63: UserWarning: Geodesic distance module is unavailable; some functionality for surfaces will be unavailable.
  warnings.warn(msg)


In [2]:
data_folder = os.path.join(os.getcwd(), 'data')
exp_name = '20220523_trial0'
exp_folder = os.path.join(data_folder, exp_name)
os.makedirs(exp_folder, exist_ok=True)
# os.environ['RAY_PICKLE_VERBOSE_DEBUG']='1'

In [3]:
config={ "c_ee": tune.uniform(12.5, 13.5),
        "c_ei": tune.uniform(7, 10),
        'c_ie': tune.uniform(6, 18),
        'c_ii': tune.uniform(7, 10),
        'tau_e': tune.uniform(0, 150),
        'tau_i': tune.uniform(0, 150),
        'r_e': tune.uniform(0.5, 2),
        'r_i': tune.uniform(0.5, 2)
        }

def run_model(config):
    params = config_params({"c_ee":config['c_ee'],'c_ei':config['c_ei'],'c_ie':config['c_ie'],'c_ii':config['c_ii'],
                   'tau_e':config['tau_e'],'tau_i':config['tau_i'],'r_e':config['r_e'],'r_i':config['r_i']})
    region = config_one_region()
    surface = config_surface(region)
    sim = config_simulator(params, region, surface)
    result_name, oscillation = run_simulation(sim, data_folder, params)
    score = evaluation(result_name, oscillation)
    tune.report(score=score)
# def run_model(config):
#     params = config_params({"c_ee":config['c_ee'],'c_ei':config['c_ei'],'c_ie':config['c_ie'],'c_ii':config['c_ii'],
#                    'tau_e':config['tau_e'],'tau_i':config['tau_i'],'r_e':config['r_e'],'r_i':config['r_i']})
#     result_name, oscillation = try_model(params, data_folder)
#     score = evaluation(result_name, oscillation)
#     tune.report(score)

In [4]:
# # Customize the logger
# from typing import Dict, List
# import json
# from ray.tune.logger import LoggerCallback

# class CustomLoggerCallback(LoggerCallback):
#     """Put the results of all trails in the same file"""

#     def __init__(self, filename: str = "log.txt"):
#         self._filename = filename
#         self._filepath = os.path.join(exp_folder, self._filename)
        
#     def log_trial_start(self, trial: "Trial"):
#         self._file = open(self._filepath, "at")
#         self._file.write(json.dumps(trial.evaluated_params))

#     def log_trial_result(self, iteration: int, trial: "Trial", result: Dict):
#         self._file = open(self._filepath, "at")
#         try:
#             self._file.write(json.dumps(result))
#         except TypeError:
#             for idx, (k, v) in enumerate(result.items()):
#                 if isinstance(v, (np.ndarray,)):
#                     result[k] = v.tolist()
#                 print(result[k])
#                 print(type(result[k]))
#                 sys.stdout.flush()
#             self._file.write(json.dumps(result))

#     def on_trial_complete(self, iteration: int, trials: List["Trial"],
#                           trial: "Trial", **info):
#         self._file = open(self._filepath, "at")
#         self._file.write(str(datetime.datetime.now())+'\n')
#         self._file.close()

In [ ]:
smoke_test = True
nsample = 10 # 1000
bayesopt = BayesOptSearch(metric="score", mode="min")
analysis = tune.run(run_model,
                    verbose=3,
                    name = exp_name,
                    local_dir=os.path.join(data_folder,'ray_results'),
                    search_alg=bayesopt,
                    config=config,
                    metric="score",
                    mode="min",
                    progress_reporter=tune.JupyterNotebookReporter(overwrite=True,metric_columns='score',
                                                                  parameter_columns=config),
                    # callbacks=[CustomLoggerCallback("log_test.txt")],
                    num_samples=2 if smoke_test else nsample)

Trial name,status,loc,<ray.tune.sample.Float object at 0x7fdb4771dac0>,<ray.tune.sample.Float object at 0x7fdb4771db50>,<ray.tune.sample.Float object at 0x7fdb4771dbe0>,<ray.tune.sample.Float object at 0x7fdb4771dc70>,<ray.tune.sample.Float object at 0x7fdb4771dd00>,<ray.tune.sample.Float object at 0x7fdb4771dd90>,<ray.tune.sample.Float object at 0x7fdb4771de20>,<ray.tune.sample.Float object at 0x7fdb4771deb0>
run_model_6c7a9b8c,RUNNING,10.131.1.45:39197,12.8745,9.85214,14.7839,8.79598,8.71254,129.926,0.734028,0.733992
run_model_6faa9a5a,RUNNING,10.131.1.45:39229,13.1011,9.12422,6.24701,9.90973,27.2737,27.5107,1.74866,0.818509


In [7]:
print("Best hyperparameters found were: ", analysis.best_config)
print("Best result: ", analysis.best_result)

2022-05-24 07:06:49,919	WARNING experiment_analysis.py:582 -- Could not find best trial. Did you pass the correct `metric` parameter?
2022-05-24 07:06:49,921	WARNING experiment_analysis.py:582 -- Could not find best trial. Did you pass the correct `metric` parameter?


Best hyperparameters found were:  None


AttributeError: 'NoneType' object has no attribute 'last_result'

In [8]:
analysis.results_df

/opt/app-root/src/.local/lib/python3.8/site-packages/ray/tune/analysis/experiment_analysis.py:301: UserWarning: Dataframes will use '/' instead of '.' to delimit nested result keys in future versions of Ray. For forward compatibility, set the environment variable TUNE_RESULT_DELIM='/'
  warnings.warn(


,score,time_this_iter_s,done,timesteps_total,episodes_total,training_iteration,experiment_id,date,timestamp,time_total_s,...,warmup_time,experiment_tag,config.c_ee,config.c_ei,config.c_ie,config.c_ii,config.tau_e,config.tau_i,config.r_e,config.r_i
trial_id,,,,,,,,,,,,,,,,,,,,,
6c7a9b8c,[0.3538349270735989],772.053930,True,None,None,1,0fe18022df4044cb9fab69e075ce5f33,2022-05-23_21-33-33,1653334413,772.053930,...,0.007703,"1_c_ee=12.875,c_ei=9.8521,c_ie=14.784,c_ii=8.7...",12.874540,9.852143,14.783927,8.795975,8.712542,129.926422,0.734028,0.733992
6faa9a5a,[0.3654444572706151],779.569084,True,None,None,1,2738ce243a9c428599e05fa775087230,2022-05-23_21-33-46,1653334426,779.569084,...,0.007398,"2_c_ee=13.101,c_ei=9.1242,c_ie=6.247,c_ii=9.90...",13.101115,9.124218,6.247014,9.909730,27.273745,27.510676,1.748664,0.818509


In [ ]:
# params = config_params()
# region = config_one_region()
# surface = config_surface(region)
# sim = config_simulator(params, region, surface)
# result_name, oscillation = run_simulation(sim)
# score = evaluation(result_name, oscillation)

In [ ]:
# def try_model(params, data_folder):
#     from datetime import date
#     k_e = np.array([1.0])            #Max value of excitatory response function
#     k_i = np.array([2.0])            #Max value of inhibitory response function
#     r_e = np.array([1.0])            #Excitatory refractory period
#     r_i = np.array([1.0])            #Inhibitory refractory period
#     c_ee = np.array([13.25])          #Excitatory to excitatory coupling coefficient
#     c_ei = np.array([12.0])          #Inhibitory to excitatory coupling coefficient
#     c_ie = np.array([9.23])         #Excitatory to inhibitory coupling coefficient
#     c_ii = np.array([2.0])          #Inhibitory to inhibitory coupling coefficient
#     tau_e = np.array([10.0])         #Membrane time-constant for the excitatory population
#     tau_i = np.array([10.0])         #Membrane time-constant for the inhibitory population
#     a_e = np.array([1.3])            #Slope parameter for the excitatory response function
#     b_e = np.array([2.8])           #Position of the maximum slope of the excitatory sigmoid function
#     c_e = np.array([7.0])            #Amplitude parameter for the excitatory response function
#     a_i = np.array([2.0])            #Slope parameter for the inhibitory response function
#     b_i = np.array([4.0])            #Position of the maximum slope of the inhibitory sigmoid function
#     c_i = np.array([1.0])            #Amplitude parameter for the inhibitory response function
#     theta_e = np.array([2.0])        #Excitatory treshold
#     theta_i = np.array([1.5])        #Inhibitory treshold
#     alpha_e = np.array([1.0])        # ?Balance parameter between excitatory and inhibitory masses
#     alpha_i = np.array([1.0])        # ?Balance parameter between excitatory and inhibitory masses
#     P = np.array([2.5])              #External stimulus to the excitatory population
#     Q = np.array([0.0])              #External stimulus to the inhibitory population
#     shift_sigmoid = np.array([False])

#     DEFAULT_PARAMS = {'k_e':k_e, 'k_i':k_i, 'r_e':r_e, 'r_i':r_i, 'c_ee':c_ee, 'c_ei':c_ei,
#                       'c_ie':c_ie, 'c_ii':c_ii, 'tau_e':tau_e, 'tau_i':tau_i,
#                       'a_e':a_e, 'b_e':b_e, 'c_e':c_e, 'a_i':a_i, 'b_i':b_i, 'c_i':c_i,
#                       'theta_e':theta_e, 'theta_i':theta_i, 'alpha_e':alpha_e, 'alpha_i':alpha_i,
#                       'P':P, 'Q':Q, 'shift_sigmoid':shift_sigmoid}
#     # update the params
#     if params:
#         for i,j in enumerate(params.keys()):
#             if j in DEFAULT_PARAMS:
#                 DEFAULT_PARAMS[j] = np.array(params[j])
#     white_matter = connectivity.Connectivity.from_file()
#     tract_lengths = white_matter.tract_lengths[73:74,73:74]
#     region_labels = white_matter.region_labels[73:74]
#     centres = white_matter.centres[73:74]
#     cortical = white_matter.cortical[73:74]
#     orientations = white_matter.orientations[73:74]
#     areas = white_matter.areas[73:74]
#     weights = white_matter.weights[73:74,73:74]
#     one_region = connectivity.Connectivity(number_of_regions = 1,
#                                             tract_lengths = tract_lengths,
#                                             region_labels = region_labels,
#                                             centres = centres,
#                                             cortical = cortical,
#                                             orientations = orientations,
#                                             areas = areas,weights = weights)
#     one_region.configure()
#     source_path = '/mnt/user/drive/My Libraries/tutorials&explorations/data/cortex/cortex_180.zip'
#     mapping_path = '/mnt/user/drive/My Libraries/tutorials&explorations/data/cortex/regionMapping_180_1.txt'
#     local_connectivity_path = '/mnt/user/drive/My Libraries/tutorials&explorations/data/cortex/local_connectivity_180.mat'
#     default_cortex = cortex.Cortex.from_file(source_file=source_path, region_mapping_file=mapping_path)
#     default_cortex.coupling_strength = np.array([2**-10])
#     default_cortex.local_connectivity = local_connectivity.LocalConnectivity.from_file(local_connectivity_path)
#     default_cortex.region_mapping_data.connectivity = one_region
#     default_cortex.configure()
    
#     mod = models.WilsonCowan(k_e = params['k_e'],            #Max value of excitatory response function
#                      k_i = params['k_i'],            #Max value of inhibitory response function
#                      r_e = params['r_e'],            #Excitatory refractory period
#                      r_i = params['r_i'],            #Inhibitory refractory period
#                      c_ee = params['c_ee'],          #Excitatory to excitatory coupling coefficient
#                      c_ei = params['c_ei'],          #Inhibitory to excitatory coupling coefficient
#                      c_ie = params['c_ie'],         #Excitatory to inhibitory coupling coefficient
#                      c_ii = params['c_ii'],          #Inhibitory to inhibitory coupling coefficient
#                      tau_e = params['tau_e'],         #Membrane tie-constant for the excitatory population
#                      tau_i = params['tau_i'],         #Membrane time-constant for the inhibitory population
#                      a_e = params['a_e'],            #Slope parameter for the excitatory response function
#                      b_e = params['a_i'],           #Position of the maximum slope of the excitatory sigmoid function
#                      c_e = params['c_e'],            #Amplitude parameter for the excitatory response function
#                      a_i = params['a_i'],            #Slope parameter for the inhibitory response function
#                      b_i = params['b_i'],            #Position of the maximum slope of the inhibitory sigmoid function
#                      c_i = params['c_i'],            #Amplitude parameter for the inhibitory response function
#                      theta_e = params['theta_e'],        #Excitatory treshold
#                      theta_i = params['theta_i'],        #Inhibitory treshold
#                      alpha_e = params['alpha_e'],        # ?Balance parameter between excitatory and inhibitory masses
#                      alpha_i = params['alpha_i'],        # ?Balance parameter between excitatory and inhibitory masses
#                      P = params['P'],              #External stimulus to the excitatory population
#                      Q = params['Q'],              #External stimulus to the inhibitory population
#                      shift_sigmoid = params['shift_sigmoid']
#                     )
#     mon_savg = monitors.SpatialAverage(period=2**-1)

#     sim = simulator.Simulator(model = mod, connectivity = one_region,
#                               coupling = c, 
#                               integrator = integ, monitors = (mon_savg,),
#                               simulation_length = 100000,
#                               surface = default_cortex)

#     sim.configure()

#     savg_data = []
#     savg_time = []
#     batch = 0
#     last_t = []
#     latt_d = []
#     check_point = 50000
#     for savg in sim():
#         # if not tavg is None:
#             # tavg_time.append(tavg[0])
#             # tavg_data.append(tavg[1])
#         if not savg is None:
#             savg_time.append(savg[0][0])
#             savg_data.append(savg[0][1])
#             if len(savg_time) == check_point:
#                 SAVG = np.array(savg_data)
#                 v1 = SAVG[:,0,0,0]
#                 if batch == 1:
#                     if sum(abs(v1[-10000:])) < 800:
#                         print("Oscillation is not generated. sum of all time point:%d. Will abandon this trail."%SUM(abs(v1[-10000:])))
#                         plot_result(savg_time,v1,[40000,49999])
#                         break
#                     else:
#                         oscillation = True
#                 batch +=1
#                 last_t = savg_time
#                 last_d = savg_data
#                 df= []
#                 savg_data = []
#                 savg_time = []
#     return (result_name, oscillation)